In [1]:
# For some reasons, file descriptors (FDs) do not get released.
# This is a work around which increases the allowed limit.
import resource
rlimit = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (2048, rlimit[1]))

In [2]:
import math
import os
import re
import random
import multiprocessing as mp
import numpy as np
from scipy.io import loadmat
import torch
import pytorch_lightning as pl
from copy import deepcopy
from sklearn.metrics import roc_auc_score, average_precision_score
from torch.optim.swa_utils import AveragedModel, SWALR

/Users/ahmedmohammed/opt/miniconda3/envs/pytorch_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Batch size
train_B = 16
val_B = 1

montage = 'BH'
p_dropout = 0.

window_len = 50
#overlap = 0.8
overlap = 0.5
sample_H = 3
sample_d_qk = 5
sample_d_v = 5
mid_features = 15
epoch_H = 3
epoch_d_qk = 5
epoch_d_v = 5

SCALE = True
data_folder = 'BH average 100-sps (0.75-38 Hz)'
files_names = os.listdir(data_folder)

g1 = ['PAT31', 'MC', 'FEsg', 'DHut', 'NKra', 'PAT2']
g2 = ['RC', 'MBra', 'ESow', 'PAT1', 'EG', 'FigSa', 'PAT28']
g3 = ['PAT47', 'RA', 'PAT51', 'MPi', 'GNA', 'LM', 'DG']
g4 = ['PAT33', 'PB', 'PAT22', 'MAXJ', 'PAT19', 'LP', 'HeMod', 'PAT50', 'PJuly']
g5 = ['PAT48', 'KS', 'PAT3', 'PAT8', 'LRio', 'HB', 'ALo', 'JAlv', 'PAT25', 'AR']

In [ ]:
train_patients = g2 + g3 + g4 + g5
val_patients = g1
del g1, g2, g3, g4, g5

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, segments, labels, flip_prob, montage):
        self.segments = segments
        self.labels = labels
        self.flip_prob = flip_prob
        
        if flip_prob > 0:
            self.idx_swaps = self.montage_swap_func(montage)
    
    
    @staticmethod
    def montage_swap_func(montage):
        if montage == 'BH':
            electrodes = ['Fp1','F7','T3','T5','O1','F3','C3','P3','Fz','Cz','Pz','Fp2','F8','T4','T6','O2','F4','C4','P4']
        elif montage == 'MCH':
            electrodes = ['C3','C4','O1','O2','Cz','F3','F4','F7','F8','Fz','Fp1','Fp2','P3','P4','Pz','T3','T4','T5','T6']
        else:
            raise NameError('Unavailable Montage')
        
        dic = {electrode: idx for idx, electrode in enumerate(electrodes)}
        
        label_swaps = [('Fp1', 'Fp2'), ('F7', 'F8'), ('F3', 'F4'), ('C3', 'C4'), ('P3', 'P4'), ('T3', 'T4'), ('T5', 'T6'), ('O1', 'O2')]
        return [(dic[e_left], dic[e_right]) for e_left, e_right in label_swaps]
    
    
    def flip_lr(self, segment):
        # flip based on self.idx_swaps
        for e_left, e_right in self.idx_swaps:
            cloned_channel = torch.clone(segment[e_left])
            segment[e_left] = segment[e_right]
            segment[e_right] = cloned_channel
        
        return segment
    
    
    def __len__(self):
        return len(self.labels)
    
    
    def __getitem__(self, index):
        if random.random() < self.flip_prob:
            return self.flip_lr( torch.clone(self.segments[index]) ), self.labels[index]
        else:
            return self.segments[index], self.labels[index]
        

In [4]:
def prep_patient_data(patient_name):
    
    patient_segments = []
    patient_labels = []
    
    regex = re.compile(f'{patient_name}\D[0-9.a-z_A-Z]*')
    
    for file_name in files_names:
        if regex.match(file_name):
            segment = torch.tensor( loadmat(os.path.join(data_folder, file_name))['segment'] ).float()
            if 'sp' in file_name:
                label = torch.tensor(1.0).float()
            elif 'ns' in file_name:
                label = torch.tensor(0.0).float()
            
            patient_segments.append(segment)
            patient_labels.append(label)
    
    return patient_segments, patient_labels


In [5]:
def prepare_dataset(patients_names):
    group_segments = []
    group_labels = []
    
    #with mp.Pool(processes=mp.cpu_count() - 1) as p:
    #    for patient_segments, patient_labels in p.imap_unordered(prep_patient_data, patients_names):
    #        group_segments += patient_segments
    #        group_labels += patient_labels
    
    for patient_name in patients_names:
        patient_segments, patient_labels = prep_patient_data(patient_name)
        group_segments += patient_segments
        group_labels += patient_labels
    
    return group_segments, group_labels


In [ ]:
train_segments, train_labels = prepare_dataset(train_patients)
val_segments, val_labels = prepare_dataset(val_patients)


In [6]:
def median_IQR_func(group_segments):
    numbers = []
    for segment in group_segments:
        for row in range(segment.shape[0]):
            for col in range(segment.shape[1]):
                numbers.append(segment[row, col].item())
    
    quantiles = torch.tensor(numbers).quantile( torch.tensor([0.25, 0.5, 0.75]) )
    median = quantiles[1].item()
    IQR = quantiles[2].item() - quantiles[0].item()
    
    return median, IQR


In [ ]:
def scaler_func(group_segments, median, IQR):
    # Scales the group_segments list inplace
    for idx in range(len(group_segments)):
        group_segments[idx] = ((group_segments[idx] - median) / IQR)
    
    return group_segments


In [ ]:
if SCALE:
    median, IQR = median_IQR_func(train_segments)
    train_segments = scaler_func(train_segments, median, IQR)
    val_segments = scaler_func(val_segments, median, IQR)

#train_dataset = CustomDataset(train_segments, train_labels, 0.5, 'BH')
#val_dataset = CustomDataset(val_segments, val_labels, 0., 'BH')


In [ ]:
del files_names, train_patients, val_patients


In [7]:
class PositionalEncoding(torch.nn.Module):
    def __init__(self, base):
        super().__init__()
        self.base = base
    
    
    def forward(self, x):
        # x: (B, C=no_input_features, T)
        _, d, T = x.shape
        
        pe = torch.zeros((d, T), device=x.device)
        t = torch.arange(T, device=x.device).reshape(1, T)
        
        col_len = math.ceil(d / 2)
        col = torch.arange(start=1, end=col_len+1, device=x.device).reshape(col_len, 1)
        
        angle = t / (self.base ** ((2 * col) / d))
        sin = torch.sin(angle)
        
        if d % 2 == 0:
            cos = torch.cos(angle)
        else:
            cos = torch.cos(angle[0:-1])
        
        #cos = torch.where(torch.tensor(d % 2 == 0, device=x.device), torch.cos(angle), torch.cos(angle[0:-1]))
        
        pe[0::2, :] = sin
        pe[1::2, :] = cos
        
        # return shape: (B, C=no_input_features, T)
        return pe + x
    

In [8]:
class MaskedMultiHeadSelfAttention(torch.nn.Module):
    def __init__(self, no_input_features, H, d_qk, d_v, bias_att=True):
        super().__init__()
        self.H = H
        self.d_qk = d_qk
        self.d_v = d_v
        
        self.q = torch.nn.Parameter(torch.empty(H, d_qk))
        torch.nn.init.xavier_normal_(self.q)
        
        self.att_lin = torch.nn.Linear(
            in_features=no_input_features,
            out_features=(H * d_qk) + (H * d_v),
            bias=bias_att
        )
    
    
    
    @staticmethod
    def qkv_func(q, k, v):
        # num_freq_bands: F or E or T
        # q: (B, H, 1, d_qk)
        # k: (B, H, F or E or T, d_qk)
        # v: (B, H, F or E or T, d_v)
        
        B, H, T, d_qk = q.shape
        d_v = v.shape[-1]
        
        # logits: (B, H, 1, T)
        logits = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_qk)     # logits: (B, H, 1, T)
        s_m = logits.softmax(dim=-1)                                        # s_m: (B, H, 1, T)
        
        # (B, H, 1, T) * (B, H, T, d_v) = (B, H, 1, d_v) ===permute===> (B, 1, H, d_v) ===reshape===> (B, H * d_v)
        return torch.matmul(s_m, v).transpose(-2,-3).reshape((B, H * d_v))
    
    
    
    def forward(self, x, dim):
        if dim not in {-1, -2}:
            raise NameError(f'Error: dim ({dim}) has to be either -1 or -2 !')
        
        if dim == -1:
            # x: (B, C=no_input_features, T)
            x.transpose_(-1, -2)
        
        # x: (B, T, C=no_input_features)
        B, T, _ = x.shape
        
        k, v = self.att_lin(x).split(split_size=[self.H * self.d_qk, self.H * self.d_v], dim=-1)
        
        q = self.q.expand(B, 1, self.H, self.d_qk).transpose(-2, -3)     # q: (B, self.H, 1, self.d_qk)
        k = k.reshape((B, T, self.H, self.d_qk)).transpose(-2, -3)       # k: (B, self.H, T, self.d_qk)
        v = v.reshape((B, T, self.H, self.d_v)).transpose(-2, -3)        # v: (B, self.H, T, self.d_v)
        
        qkv = self.qkv_func(q, k, v)                                     # qkv: (B, self.H * self.d_v)
        
        if dim == -1:
            x.transpose_(-1, -2)                                         # (B, C=no_input_features, T)
        
        return qkv
    

In [ ]:
class CosineWarmupScheduler(torch.optim.lr_scheduler._LRScheduler):

    def __init__(self, optimizer, warmup, max_num_iters):
        self.warmup = warmup
        self.max_num_iters = max_num_iters
        super().__init__(optimizer)
    
    
    def get_lr(self):
        # returns a list of the learning rates
        lr_factor = self.get_lr_factor(epoch=self.last_epoch)
        return [base_lr * lr_factor for base_lr in self.base_lrs]
    
    
    def get_lr_factor(self, epoch):
        # Optional method that computes lr_factor, NOT learning rate itself.
        if epoch <= self.warmup:
            slope = (1 - 0) / (self.warmup - 0)
            lr_factor = slope * epoch
        else:
            f = 1 / (2 * (self.max_num_iters - self.warmup))
            lr_factor = 0.5 * ( 1 + math.cos( 2 * math.pi * f * (epoch - self.warmup) ) )
        
        return lr_factor
    

In [ ]:
min_validation_loss = math.inf
best_epoch = 0
best_model_state = None
AP_at_min_val_loss = 0.


In [9]:
# create the pytorch lightning module
class CustomLightningModule(pl.LightningModule):
    def __init__(self, p_dropout, window_len, overlap, sample_H, sample_d_qk, sample_d_v, mid_features, epoch_H, epoch_d_qk, epoch_d_v):
        super().__init__()
        
        self.window_len = window_len
        self.increment = round((1 - overlap) * window_len)
        
        
        self.sample_pe = PositionalEncoding(100)
        
        self.sample_attention = torch.nn.ModuleList()
        self.sample_nonlinearity = torch.nn.ModuleList()
        for end_idx in range(window_len - 1, 300, self.increment):
            att = MaskedMultiHeadSelfAttention(19, sample_H, sample_d_qk, sample_d_v)
            self.sample_attention.append(att)
            
            non_linear = torch.nn.Sequential(
                torch.nn.Linear(in_features=sample_H*sample_d_v, out_features=mid_features),
                torch.nn.LeakyReLU(negative_slope=0.01),
                torch.nn.Dropout(p=p_dropout)
            )
            self.sample_nonlinearity.append(non_linear)
            
        
        if end_idx != 299:
            raise NameError(f'Error: Inspect carefully the overlap ({overlap}) argument!')
        
        
        self.epoch_pe = PositionalEncoding(100)
        self.epoch_att = MaskedMultiHeadSelfAttention(mid_features, epoch_H, epoch_d_qk, epoch_d_v)
        
        self.seq = torch.nn.Sequential(
            torch.nn.Linear(in_features=epoch_H*epoch_d_v, out_features=10),
            torch.nn.LeakyReLU(negative_slope=0.01),
            torch.nn.Dropout(p=p_dropout),
            torch.nn.Linear(in_features=10, out_features=5),
            torch.nn.LeakyReLU(negative_slope=0.01),
            torch.nn.Dropout(p=p_dropout),
            torch.nn.Linear(in_features=5, out_features=1)
        )
        
        
        self.criterion = torch.nn.BCEWithLogitsLoss()
        self.automatic_optimization = False
    
    
    
    
    def forward(self, x):
        # x: (B, 19, T)
        
        lis = []
        for i, (att, non_linear) in enumerate(zip(self.sample_attention, self.sample_nonlinearity)):
            start_idx = i * self.increment
            end_idx = start_idx + self.window_len
            
            encoded_epoch = self.sample_pe(x[:, :, start_idx:end_idx])
            attention_output = att(encoded_epoch, -1)            # (B, sample_H * sample_d_v)
            
            non_linear_output = non_linear(attention_output)     # (B, mid_features)
            lis.append(non_linear_output)
        
        
        epochs = torch.stack(lis, -1)                     # (B, mid_features, no_epochs)
        epochs = self.epoch_pe(epochs)                    # (B, mid_features, no_epochs)
        attention_output = self.epoch_att(epochs, -1)     # (B, epoch_H * epoch_d_v)
        
        logits = self.seq(attention_output)             # (B, 1)
        
        return logits.squeeze(dim=-1)
    
    
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), weight_decay=0)
        cosine_scheduler = CosineWarmupScheduler(optimizer=optimizer, warmup=100, max_num_iters=self.trainer.max_epochs)
        swa_scheduler = SWALR(optimizer, swa_lr=0.0001, anneal_epochs=100, anneal_strategy='cos')
        return [optimizer], [cosine_scheduler, swa_scheduler]
    
    
    
    def train_dataloader(self):
        dataset = CustomDataset(train_segments, train_labels, 0.5, montage)
        # mp.cpu_count()-1
        return torch.utils.data.DataLoader(dataset, batch_size=train_B, shuffle=True, num_workers=0,
                                           pin_memory=True, drop_last=True)
    
    
    
    def val_dataloader(self):
        dataset = CustomDataset(val_segments, val_labels, 0, montage)
        # mp.cpu_count()-1
        return torch.utils.data.DataLoader(dataset, batch_size=val_B, shuffle=False, num_workers=0,
                                           pin_memory=True, drop_last=False)
    
    
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        
        self.eval()
        with torch.no_grad():
            logits_before_optimizer_step = self(x)
            loss_before_optimizer_step = self.criterion(logits_before_optimizer_step, y)
        self.train()
        
        
        opt = self.optimizers()

        def closure():
            logits = self(x)
            loss = self.criterion(logits, y)
            opt.zero_grad()
            self.manual_backward(loss)
            return loss

        opt.step(closure=closure)
        
        return {'loss_before_optimizer_step': loss_before_optimizer_step.detach()}
    
    
    
    def training_epoch_end(self, training_step_outputs):
        if self.global_rank == 0:
            train_loss = torch.stack([dic['loss_before_optimizer_step'] for dic in training_step_outputs]).mean()
            print(f'Epoch: {self.current_epoch}')
            print(f'Training loss BEFORE optimizer step: {round(train_loss.item(), 5)}')
            print('--------------------------------------')
    
    
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        return {'loss': loss, 'y': y, 'logits': logits}
    
    
    
    def validation_epoch_end(self, validation_step_outputs):
        global min_validation_loss, best_epoch, best_model_state, AP_at_min_val_loss
        
        swa_start_epoch = 1000
        
        cosine_scheduler, swa_scheduler = self.lr_schedulers()
        
        if self.current_epoch > 0:
            if self.current_epoch < swa_start_epoch:
                cosine_scheduler.step()
            else:
                swa_scheduler.step()
            
        
        
        if self.global_rank == 0:
            
            if self.current_epoch > (swa_start_epoch + swa_scheduler.anneal_epochs):
                swa_model.update_parameters(self)
                
            
            val_loss = torch.stack([dic['loss'] for dic in validation_step_outputs]).mean()
            val_loss = val_loss.item()
            print(f'val_loss: {val_loss}')
            
            y = torch.cat([dic['y'] for dic in validation_step_outputs]).cpu()
            logits = torch.cat([dic['logits'] for dic in validation_step_outputs]).cpu()
            pred = torch.sigmoid(logits)
            
            y = np.array(y)
            pred = np.array(pred)
            
            try:
                validation_AP = average_precision_score(y, pred)
                
                if (val_loss < min_validation_loss) and (self.current_epoch > 1):
                    min_validation_loss = val_loss
                    best_epoch = self.current_epoch
                    best_model_state = deepcopy(self.state_dict())
                    AP_at_min_val_loss = validation_AP
                
                
                print(f'validation_AP = {round(validation_AP, 5)} ... AP_at_min_val_loss = {round(AP_at_min_val_loss, 5)} @ epoch {best_epoch}')
                
            except ValueError:
                print(f'ValueError @ epoch: {self.current_epoch} ====> y = {y}')
            
            print('=======================================================================================')


In [ ]:
my_lightning_module = CustomLightningModule(p_dropout, window_len, overlap, sample_H, sample_d_qk,
                                            sample_d_v, mid_features, epoch_H, epoch_d_qk, epoch_d_v)

swa_model = AveragedModel(my_lightning_module)

trainer = pl.Trainer(gpus=0, enable_checkpointing=False, enable_progress_bar=False, logger=False, max_epochs=400)

trainer.fit(my_lightning_module)

# Update bn statistics for the swa_model at the end
torch.optim.swa_utils.update_bn(my_lightning_module.train_dataloader(), swa_model)


In [ ]:
PATH = f'models/g1_hierarchical_attention_10k_AP_{round(100 * AP_at_min_val_loss, 2)}.pt'
torch.save(best_model_state, PATH)

In [10]:
import pandas as pd


model = CustomLightningModule(p_dropout, window_len, overlap,
                              sample_H, sample_d_qk, sample_d_v,
                              mid_features, epoch_H, epoch_d_qk, epoch_d_v)

#IEDs_names = [item for item in os.listdir(data_folder) if 'sp' in item]
NIEDs_names = [item for item in os.listdir(data_folder) if 'ns' in item]


In [11]:
lis = [item for item in os.listdir('models/') if 'g1_hierarchical_attention' in item]
g1_model_name = max(lis)
train_segments, _ = prepare_dataset(g2 + g3 + g4 + g5)
g1_median, g1_IQR = median_IQR_func(train_segments)

lis = [item for item in os.listdir('models/') if 'g2_hierarchical_attention' in item]
g2_model_name = max(lis)
train_segments, _ = prepare_dataset(g1 + g3 + g4 + g5)
g2_median, g2_IQR = median_IQR_func(train_segments)


lis = [item for item in os.listdir('models/') if 'g3_hierarchical_attention' in item]
g3_model_name = max(lis)
train_segments, _ = prepare_dataset(g1 + g2 + g4 + g5)
g3_median, g3_IQR = median_IQR_func(train_segments)


lis = [item for item in os.listdir('models/') if 'g4_hierarchical_attention' in item]
g4_model_name = max(lis)
train_segments, _ = prepare_dataset(g1 + g2 + g3 + g5)
g4_median, g4_IQR = median_IQR_func(train_segments)


lis = [item for item in os.listdir('models/') if 'g5_hierarchical_attention' in item]
g5_model_name = max(lis)
train_segments, _ = prepare_dataset(g1 + g2 + g3 + g4)
g5_median, g5_IQR = median_IQR_func(train_segments)


#s = pd.Series(index=IEDs_names, dtype='float64')
s = pd.Series(index=NIEDs_names, dtype='float64')


#for IED_name in IEDs_names:
for NIED_name in NIEDs_names:
    #segment = loadmat(os.path.join(data_folder, IED_name))['segment']
    segment = loadmat(os.path.join(data_folder, NIED_name))['segment']
    segment = torch.tensor(segment).float()
    
    #patient_name = IED_name[: IED_name.find('_')]
    patient_name = NIED_name[: NIED_name.find('_')]
    
    if patient_name in g1:
        segment = (segment - g1_median) / g1_IQR
        model.load_state_dict(torch.load(f'models/{g1_model_name}'))
    elif patient_name in g2:
        segment = (segment - g2_median) / g2_IQR
        model.load_state_dict(torch.load(f'models/{g2_model_name}'))
    elif patient_name in g3:
        segment = (segment - g3_median) / g3_IQR
        model.load_state_dict(torch.load(f'models/{g3_model_name}'))
    elif patient_name in g4:
        segment = (segment - g4_median) / g4_IQR
        model.load_state_dict(torch.load(f'models/{g4_model_name}'))
    elif patient_name in g5:
        segment = (segment - g5_median) / g5_IQR
        model.load_state_dict(torch.load(f'models/{g5_model_name}'))
    else:
        raise NameError('Unknown Patient')
    
    
    segment.unsqueeze_(0)
    model.eval()
    
    #s.at[IED_name] = torch.sigmoid(model(segment)).item()
    s.at[NIED_name] = torch.sigmoid(model(segment)).item()

In [12]:
s

RC_24_ns.mat      0.002088
FEsg_38_ns.mat    0.013555
HB_2_ns.mat       0.010820
FEsg_26_ns.mat    0.218849
FigSa_9_ns.mat    0.083698
                    ...   
LM_3_ns.mat       0.047767
RC_6_ns.mat       0.009134
FEsg_19_ns.mat    0.042846
NKra_10_ns.mat    0.115910
PAT48_ns_6.mat    0.001987
Length: 593, dtype: float64

In [13]:
#with pd.ExcelWriter('IEDs_results.xlsx', mode='a') as writer:
with pd.ExcelWriter('NIEDs_results.xlsx', mode='a') as writer:
    s.to_excel(writer, sheet_name='hierarchical_attention', header=False)